In [32]:
import cv2
import numpy as np
import random
import os
from tqdm import tqdm  #跑時間條的library

from random import sample


In [33]:
def rotate_bound(image, angle):
    """

    :param image: 原圖
    :param angle: 旋轉角度
    :return: 旋轉後的圖
    """
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
    
    img = cv2.warpAffine(image, M, (nW, nH))
    # perform the actual rotation and return the image
    return img

In [80]:
class com:
    def __init__(self, background_list, cut_list,bg_name,label_name):  
        self.background_list = background_list
        self.cut_list = cut_list
        self.bg_name  = bg_name
        self.label_name = label_name

        
    def back_init(self, file_name):    #重新讀取新的背景圖，一些參數更新
#         self.background = cv2.imread("./background/"+background_list[random.randint(0,11)])
        name = sample(self.background_list,1)
#         print(self.bg_name+name[0])
        self.background = cv2.imread(self.bg_name+name[0])
        self.Xrange = []
        self.Yrange = []
        self.file_name = file_name
        
    def cut_init(self,j):   #重新讀取新的裁切圖
        self.classid = sample(self.cut_list,1)
#         self.classid = random.randint(0,7)
        self.cut = cv2.imread(self.label_name+self.classid[0])
        print(self.label_name+self.classid[0])
        self.shape = self.cut.shape
        print(self.shape)
        self.cut = cv2.resize(self.cut,(self.shape[0]//2,self.shape[1]//2))
        
    def combine(self):     #合成
        angle = random.randrange(-10,10)
        rimg = rotate_bound(self.cut, angle)
        shape = self.background.shape
        
        shift_x = random.randint(310,720) - rimg.shape[0]//2    #120,320 120,320 320,520 280,380 340,540  225
        shift_y = random.randint(700,1400) - rimg.shape[1]//2   #320,520 800,1000 350,550 530,730 750,950  285
        
        
        Xmin = shift_x
        Xmax = shift_x+rimg.shape[0]
        Ymin = shift_y
        Ymax = shift_y+rimg.shape[1]
        
        #if check_over: (確認重疊)
        #if self.check_cover(Xmin, Xmax, Ymin, Ymax): 
        
        if True:
            X = (Xmin + (Xmax - Xmin)/2) / self.background.shape[0]
            Y = (Ymin + (Ymax - Ymin)/2) / self.background.shape[1]
            W = (Xmax - Xmin)/self.background.shape[0]
            H = (Ymax - Ymin)/self.background.shape[1]
            #x  = 0
            
            
#             if   self.classid < 6 and self.classid >= 0 :
#                 x = 1
#             elif self.classid < 20 and self.classid >= 6 :
#                 x = 2
#             elif self.classid < 30 and self.classid >= 20 :
#                 x = 3
#             elif self.classid < 40 and self.classid >= 30 :
#                 x = 4
#             elif self.classid < 50 and self.classid >= 40 :
#                 x = 5
            

            label =str(1) + " {:.6f} {:.6f} {:.6f} {:.6f}\n".format(Y,X,H,W)    #Yolo的格式

            for i in range(rimg.shape[0]):
                for j in range(rimg.shape[1]):
                    if rimg[i,j].all() == 0:
                        pass
                    else:
                        self.background[Xmin+i, Ymin+j] = rimg[i, j]
                        
                        
            self.Xrange.append((Xmin, Xmax))  #確認重疊用的
            self.Yrange.append((Ymin, Ymax))  #確認重疊用的
            
            self.mkdir("./dataset/train/labels/")
            with open("./dataset/train/labels/"+self.file_name+".txt", "a") as f:  #寫label檔
                f.write(label)
                
    def img_write(self):    #存合成圖
        self.mkdir("./dataset/train/images/")
        cv2.imwrite("./dataset/train/images/"+self.file_name+".jpg",self.background)

    
    def mkdir(self,path):
        folder = os.path.exists(path)
        if not folder:
            os.makedirs(path)
#             print('-----建立成功-----')
        else:
            pass
#             print(path+'目錄已存在')
    
    
    
    def check_cover(self, Xmin, Xmax, Ymin, Ymax):    #確認有沒有重疊覆蓋，生成一堆的可以忽略
        
        for i in range(len(self.Xrange)):
            
            Xmin_check = 0
            Xmax_check = 0
            Ymin_check = 0
            Ymax_check = 0
            
            if (Xmin >= self.Xrange[i][0]) and (Xmin <= self.Xrange[i][1]):
                Xmin_check = 1
            if (Xmax >= self.Xrange[i][0]) and (Xmax <= self.Xrange[i][1]):
                Xmax_check = 1
            if (Ymin >= self.Yrange[i][0]) and (Ymin <= self.Yrange[i][1]):
                Ymin_check = 1
            if (Ymax >= self.Yrange[i][0]) and (Ymax <= self.Yrange[i][1]):
                Ymax_check = 1
                
            if (self.Xrange[i][0] >= Xmin) and (self.Xrange[i][0] <= Xmax):
                Xmin_check = 1
            if (self.Xrange[i][1] >= Xmin) and (self.Xrange[i][1] <= Xmax):
                Xmax_check = 1
            if (self.Yrange[i][0] >= Ymin) and (self.Yrange[i][0] <= Ymax):
                Ymin_check = 1
            if (self.Yrange[i][1] >= Ymin) and (self.Yrange[i][1] <= Ymax):
                Ymax_check = 1
            
            if (Xmin_check or Xmax_check) and (Ymin_check or Ymax_check):
               
                return False
        return True


In [82]:
background_list = []
cut_list = []
bg_name = "./dataset/bg/"
label_name = "./dataset/cut/"
for filename in os.listdir(bg_name):
    background_list.append(filename)
for filename in os.listdir(label_name):
    cut_list.append(filename)
# print(cut_list)
    
COM = com(background_list, cut_list,bg_name,label_name)
for i in tqdm(range(1,10,1)):#生成幾張
    COM.back_init(str(i))
    for j in range(1):       #一張圖裡有合成幾個裁切圖
        COM.cut_init(j)
        COM.combine()
    COM.img_write()

  0%|                                                     | 0/9 [00:00<?, ?it/s]

./dataset/cut/2.png
(506, 1027, 3)


 11%|█████                                        | 1/9 [00:00<00:02,  3.02it/s]

./dataset/cut/3.png
(508, 837, 3)


 22%|██████████                                   | 2/9 [00:00<00:02,  3.34it/s]

...

 78%|███████████████████████████████████          | 7/9 [00:02<00:00,  3.36it/s]

./dataset/cut/3.png
(508, 837, 3)


 89%|████████████████████████████████████████     | 8/9 [00:02<00:00,  3.47it/s]

./dataset/cut/4.png
(388, 951, 3)


100%|█████████████████████████████████████████████| 9/9 [00:02<00:00,  3.34it/s]


In [4]:
print(5%2)

1


In [4]:
angle = random.randrange(-10,10)
print(angle)

-3


In [17]:

image = cv2.imread("323.jpg")
(h, w) = image.shape[:2]
(cX, cY) = (w // 2, h // 2)
M = cv2.getRotationMatrix2D((cX, cY), 1 , 1)
print(M)
print(M[0,1])
cos = np.abs(M[0, 0])
sin = np.abs(M[0, 1])

# compute the new bounding dimensions of the image
nW = int((h * sin) + (w * cos))
nH = int((h * cos) + (w * sin))

# adjust the rotation matrix to take into account translation
M[0, 2] += (nW / 2) - cX
M[1, 2] += (nH / 2) - cY

img = cv2.warpAffine(image, M, (nW, nH))

[[ 0.9998477   0.01745241 -9.27808683]
 [-0.01745241  0.9998477  16.8365548 ]]
0.01745240643728351


In [69]:

def m(path):
    folder = os.path.exists(path)
    if not folder:
        os.makedirs(path)
        print('-----建立成功-----')
    else:
        print(path+'目錄已存在')
path = "./dataset/bg/"
m(path)

./dataset/bg/目錄已存在
